In [1]:
%load_ext autoreload
%cd ..

/Users/trippgordon/Desktop/Code/sonic/NoisyGAN


In [2]:
%autoreload 2
%reload_ext autoreload
%pwd
from core import *
import gin
gin.enter_interactive_mode()

In [7]:
batch_size = 8

# data_provider = FromTFRecords('./Data/test/acapella.tfrecord*')
data_provider = FromTFRecords('./Data/piano/piano30s.tfrecord*')
gan_model = GAN()
combined_iter = combined_sample_iter(gan_model.gen, data_provider, batch_size=batch_size)
opt = tf.keras.optimizers.Adam()

In [9]:
train_gan(gan_model, opt, combined_iter, model_dir="models/gan/", total_iters=1)

INFO:absl:----- GAN Step 0 -----
INFO:absl:generating...
INFO:absl:Disc loss: 5.072035Gen loss: 11.120104


In [12]:
play(gan_model.gen.generate()['audio'])